<a href="https://colab.research.google.com/github/ravichas/bioinformatics/blob/main/C11/C11_1_GeneExpr_Clustering_HC_Dentrogram_Heatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# C11 Hands-on Exercise: S.Ravichandran

In [ ]:
# based on Prof.Pevsner Bioinformatics Book
install.packages("rafalib")
library(rafalib)

In [ ]:
# prompt: download file from Github https://raw.githubusercontent.com/ravichas/bioinformatics/main/data/myarraydata.txt

z <- read.delim("https://raw.githubusercontent.com/ravichas/bioinformatics/main/data/myarraydata.txt", header = TRUE, sep="\t")


In [ ]:
z

In [ ]:
dim(z)
row.names(z) <- z[,1]
z


In genomics often cluster the data to discover groups
Also, remember that genomics data is a high-D data.
To cluster the data, we need to compute distance
we need to understand the concept of distance.

create a distance matrix using cols 3 to 16
perform a hierarchical clustering using the complete linkage
agglomeration method

HC follows either a top-down (divisive) or agglomerative (bottom-up)
approaches top-down, we take all the samples and divide them into two
(not necessarily equally) and then divide them up into two
until there are no more samples to divide

In [ ]:
clust <- hclust(dist(z[,3:16]),method="complete")
clust

plot(clust)

it generates a clustering tree
you can also repeat using methods="single" or "median"
`?hclust` for more options

create a version of matrix called z.back in which 2 columns
containing the gene names and chromosomal loci are removed

In [ ]:
z.back=z[,-c(1,2)]

z.back

# create a new file called w by transposing z.back

w <- t(z.back)

w

In [ ]:
clust <- hclust(dist(w[,1:8]))
clust
plot(clust)

In [ ]:
clust <- hclust(dist(z[,3:16],method="manhattan"),method="complete")
plot(clust)

In [ ]:
clust <- hclust(dist(z[,3:16],method="minkowski"),method="complete")
plot(clust)

In [ ]:
clust <- hclust(dist(z[,3:16],method="binary"),method="complete")
plot(clust)

In [ ]:
clust <- hclust(dist(z[,3:16],method="maximum"),method="complete")
plot(clust)

In [ ]:
clust <- hclust(dist(z[,3:16],method="canberra"),method="complete")
plot(clust)

you can vary the metric by which you create the distance matrix
Euclidean, manhattan, minkowski, binary, maximum, canberra
as well as varying the clustering method
ward, single, complete, average, mcquitty, median or centroid

In [ ]:
library(devtools)
install_github("genomicsclass/tissuesGeneExpression")
# or
# BiocManager::install("genomicsclass/tissuesGeneExpression")

In [ ]:
library(tissuesGeneExpression)
data(tissuesGeneExpression)

In [ ]:
# distance between sample1 and sample10
sqrt(sum( (e[,1]-e[,10])^2 ) )

# distance matrix of samples
distance between rows or samles (after transposing)

In [ ]:
d <- dist( t(e) )  # only lower diagonal is stored
hc <- hclust(d)
class(hc)
plot(hc) # plot knows what to do

In [ ]:
plot(hc, cex = 0.5, label = tissue) # hc traks the order of tissue

In [ ]:
myplclust(hc, cex = 0.5, label = tissue, lab.col = as.fumeric(tissue))
abline(h = 120) # let us cut the tree at 120

Remember thsee are just groupings
if you want to create clusters using the groups
then you have to decide on a cut-off and then
group the tissues based on the groups from the cut-off

How do we decide on the cut off
the most distance I can move without any interference
that is where I want to cut

In [ ]:
cl <- cutree(hc, h = 120)
table(tree=tissue, cluster=cl)

take a look at the table

Are we doing good with colon samples?
* what about liver
* what about kidney ?

# Quiz
which pair (7, 13) or (11,20) is farthest in distance?

In [ ]:
set.seed(100)
m <- 10000
n <- 24
data <- matrix(rnorm(m*n),m,n)
colnames(data) <- 1:n
d <- dist(t(data))
h <- hclust(d)
plot(h)

# k-means clustering

very easy to understand but not useful for biological data
  
we need to decide upfront how many clusters we need

then the algorithm then decides to put
which sample goes to which cluster

we pick 3 samles at ranodm we call this
3 centroids then look for close sample
then turn them into the corresponding cluster
redefine the cluster centroid and continue thsi
until there are no changes.

# Caveats:

KEEP in mind that there is a random component to this method

we do not know up-front how many clusters we expect

In [ ]:
length(unique(tissue))
km <- kmeans( t(e), 7, centers = 7 )
table(tissue, clusters=km$cluster)

# what do you think about the result?

In [ ]:
km <- kmeans( t(e), 7, centers = 7 )
table(tissue, clusters=km$cluster)

# Can we create a plot?
yes, we have to use the MDS method

In [ ]:
d <- dist( t(e) )
mds <- cmdscale( d )
plot(mds[,1], mds[,2], col=km$cluster)

In [ ]:
# kmeans using one of the microarray data
library(devtools)
install_github("genomicsclass/GSE5859Subset")
library(GSE5859Subset)
data(GSE5859Subset)
set.seed(10)
km <- kmeans( t(geneExpression), 5, centers = 5 )
table(sampleInfo$date, km$cluster)
table(sampleInfo$group, km$cluster)

In [ ]:
#  Head Map
library(devtools)
#install_github("genomicsclass/tissuesGeneExpression")
library(tissuesGeneExpression)
data(tissuesGeneExpression)

## Heat map adds structure to the data
it takes the data and clusters the gene and then it clustes the sample then it adds color yellow means high and red means low
Why
Becos, every pixel correspond to an entry in the table. we dont want to use all the rows

In [ ]:
dim(e)
# 22215 x 189
# so let us use the first 100 genes

image( e[1:100,] )

In [ ]:
# 8 minutes ** Time Consuming **
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("genefilter")

In [ ]:
library(genefilter)
rv <- genefilter::rowVars(e)
idx <- order(-rv)[1:40]
heatmap(e[idx,])

In [ ]:
# let us improve the color
install.packages("RColorBrewer")

In [ ]:
library(RColorBrewer)
hmcol <- colorRampPalette(brewer.pal(9,"GnBu"))(100)
heatmap(e[idx,],col=hmcol) # low is whie and high expr is blue
# use heatmap2 instead


In [ ]:
install.packages("gplots")

library(gplots)
library(rafalib)
cols <- palette(brewer.pal(7,"Dark2"))[as.fumeric(tissue)]
cbind(colnames(e),tissue,cols)
heatmap.2(e[idx,], labCol=tissue, trace = "none",
          ColSideColors = cols,
          col = hmcol)

# So far we have always looked at sample clustering
can we cluster gene

In [ ]:
set.seed(1)
m <- 10000
n <- 24
B <- 100
cl <- vector("numeric",B)
test <- for (i in 1:B) {
  x = matrix(rnorm(m*n),m,n)
  hc <- hclust( dist(t(x)) )
  cl[i] <- max(cutree(hc, h = 143))

}
cl
sd(cl)/sqrt(23)


Finally, note that clustering involves computing
distances and distances are very susceptible to noise


In [ ]:
session_info()